In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/artem-gordeev/ColdAtoms.jl"))

In [1]:
using ColdAtoms

In [2]:
fontsize = 14;
linewidth = 2;
color = "red";

In [20]:
using QuantumOptics
include("../params/default.jl")

N = 10;
samples, acc_rate = samples_generate(trap_params, atom_params, N; skip=5000, freq=1000);

detuning_params = [Δ0, δ_twophoton(Ωr, Ωb, Δ0)];

T0 = T_twophoton(Ωr, Ωb, Δ0);
tspan = [0.0:T0/50:6*T0;];
ψ0 = g;
blue_laser_params = [Ωb, wb, zb];

In [ ]:
ρ_mean, ρ2_mean = 
    simulation(
        tspan, ψ0, 
        atom_params, trap_params,
        samples, f,
        red_laser_phase_amplitudes,         blue_laser_phase_amplitudes,
        red_laser_params,         blue_laser_params,
        detuning_params,         decay_params;
        atom_motion = true,    free_motion = true,   
        spontaneous_decay = true, parallel = false
    );
    
Pg_gauss = real(expect(g ⊗ dagger(g), ρ_mean)); 
Pr_gauss = real(expect(r ⊗ dagger(r), ρ_mean));  

In [ ]:
using PlotlyJS
using LsqFit
@. model(x, p) = 0.5*(1 - exp(-p[2]*x)*cos(p[1]*x)) 
p0 = [18.0,1.0]
traces = GenericTrace[]

n_max = 6
for i in 0:2:n_max
    if (i == 0)
        s = 1;
    else
        s = i;
    end;
    #LG
    """blue_laser_params = [Ωb, wb/s^0.5, zb/s, "simp_flattop_LG", i, i];

    ρ_mean, ρ2_mean = simulation_blue_intens(    tspan, ψ0,      atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,       spontaneous_decay = true, parallel = false    );
    Pr_LG = real(expect(r ⊗ dagger(r), ρ_mean));  

    push!(traces, scatter(x=tspan, y=Pr_LG,    mode="lines",    name="LG_$(i)"))"""
    #HG
    #blue_laser_params = [Ωb, wb/s^0.5, zb/s, "simp_flattop_HG", i, i];
    blue_laser_params = [Ωb, wb, zb, "simp_flattop_HG", i, i];

    ρ_mean, ρ2_mean = simulation_blue_intens(    tspan, ψ0,      atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,       spontaneous_decay = true, parallel = false    );
    Pr_HG = real(expect(r ⊗ dagger(r), ρ_mean));  

    
    fit = curve_fit(model, tspan, Pr_HG, p0)
    println("HG_$(i) = ", coef(fit)[2]/coef(fit)[1])
    push!(traces, scatter(x=tspan, y=Pr_HG .+ 0.005*i,    mode="lines",    name="HG_$(i)"))
    
end;

#push!(traces, scatter(x=tspan, y=Pr_gauss,    mode="lines",    name="Gauss"))
#trace = scatter(x=x, y= abs.(E_sum).^2, mode="lines", name = "I")
plot(traces)  

In [16]:
using PlotlyJS
traces = GenericTrace[]

n_max = 8
for i in 0:2:n_max
    if (i == 0)
        s = 1;
    else
        s = i;
    end;
    #LG
    blue_laser_params = [Ωb, wb/s^0.5, zb/s, "simp_flattop_LG", i, i];

    ρ_mean, ρ2_mean = simulation_blue_intens(    tspan, ψ0,      atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,       spontaneous_decay = true, parallel = false    );
    Pr_LG = real(expect(r ⊗ dagger(r), ρ_mean));  

    
    fit = curve_fit(model, tspan, Pr_LG, p0)
    println("LG_$(i) = ", coef(fit)[2]/coef(fit)[1])
    push!(traces, scatter(x=tspan, y=Pr_LG,    mode="lines",    name="LG_$(i)"))
    #HG
    """blue_laser_params = [Ωb, wb/s^0.5, zb/s, "simp_flattop_HG", i, i];

    ρ_mean, ρ2_mean = simulation_blue_intens(    tspan, ψ0,      atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,       spontaneous_decay = true, parallel = false    );
    Pr_HG = real(expect(r ⊗ dagger(r), ρ_mean));  

    push!(traces, scatter(x=tspan, y=Pr_LG,    mode="lines",    name="HG_$(i)"))"""
    
end;

#push!(traces, scatter(x=tspan, y=Pr_gauss,    mode="lines",    name="Gauss"))
#trace = scatter(x=x, y= abs.(E_sum).^2, mode="lines", name = "I")
plot(traces)  

LG_0 = 0.0032904782287371936
LG_2 = 0.0021122853213899418
LG_4 = 0.00294976265627757
LG_6 = 0.005899593488260022
LG_8 = 0.010853009223023003


┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\А\.julia\packages\PlotlyKaleido\jXjt6\src\PlotlyKaleido.jl:38
┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\А\.julia\packages\PlotlyKaleido\jXjt6\src\PlotlyKaleido.jl:38


data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin and template"

In [ ]:
ρ_mean, ρ2_mean = simulation_blue_intens(
    tspan, ψ0,  
    atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,
    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,   
    spontaneous_decay = true, parallel = false
    );
    
Pg_HG = real(expect(g ⊗ dagger(g), ρ_mean)); 
Pr_HG = real(expect(r ⊗ dagger(r), ρ_mean));  

0.0051377421406715515

In [7]:
n=0
blue_laser_params = [Ωb, wb, zb, "simp_flattop_LG", n, n];
ρ_mean, ρ2_mean = simulation_blue_intens(
    tspan, ψ0,  
    atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,
    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,   
    spontaneous_decay = true, parallel = false
    );
    
Pg_LG = real(expect(g ⊗ dagger(g), ρ_mean)); 
Pr_LG = real(expect(r ⊗ dagger(r), ρ_mean));  

In [ ]:
using PlotlyJS
trace1 = scatter(x=tspan, y=Pr,
mode="lines+markers",
name="model HG")
trace2 = scatter(x=tspan, y=model(tspan, coef(fit)),
mode="lines+markers",
name="decaying rabi fit")
trace3 = scatter(x=tspan, y=Pr_gauss.+0.002,
mode="lines+markers",
name="model gauss")
trace4 = scatter(x=tspan, y=Pr_LG.+0.001,
mode="lines+markers",
name="model LG")
plot([trace1,trace2, trace3, trace4])   

In [5]:
using PlotlyJS
n = 8
x = -2:0.01:2

traces = GenericTrace[]
#E_sum = 0.0 .* x
for i in 0:2:n
    #c_i = HG_coeff(trunc(Int,i/2),trunc(Int,n/2)) #* HG_coeff(trunc(Int,j/2)+1,trunc(Int,m/2))
    #E_sum += c_i .* HG.(x, w, i) #.* HG.(y, w, j) .* exp.(-1.0im .* gouy) 
    if i == 0
        s = 1
    else 
        s = i
    end
    laser_params = [1, 1/s^0.5, 5, "simp_flattop", i, i];
    res = [abs(simple_flattopHG_field(xx,0,0,laser_params) .^ 2) for xx in x] 
    #res = abs.([simple_flattopHG_field(xx,0,0,laser_params) for xx in x]).^2
    push!(traces, scatter(x=x, y= res ,mode="lines", name = "HG_$(i)"))

    res = [abs(simple_flattopLG_field(xx,0,0,laser_params) .^ 2) for xx in x] 
    #res = abs.([simple_flattopHG_field(xx,0,0,laser_params) for xx in x]).^2
    push!(traces, scatter(x=x, y= 0.001 .+ res ,mode="lines", name = "LG_$(i)"))
    
end;
#trace = scatter(x=x, y= abs.(E_sum).^2, mode="lines", name = "I")

layout = Layout(xaxis_title="r",
    yaxis_title="I")
plot(traces, layout)  


data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\А\.julia\packages\PlotlyKaleido\jXjt6\src\PlotlyKaleido.jl:38
